In [56]:
import pdfplumber
import re

pdf_path = "hd_test1.pdf"

def clean_table(table):
    cleaned = []
    for row in table:
        normalized = [(cell or "").strip() for cell in row]
        cleaned.append(normalized)
    return cleaned

def remove_newlines_in_cells(table):
    cleaned = []
    for row in table:
        cleaned.append([(cell or "").replace("\n", " ").strip() for cell in row])
    return cleaned

def parse_number(value):
    if value is None:
        return None
    s = str(value).strip()
    if not re.search(r'\d', s):
        return value
    s = s.replace('.', '')
    s = s.replace(',', '.')
    if '.' in s:
        try:
            return float(s)
        except:
            return value
    try:
        return int(s)
    except:
        return value

def normalize_table_numbers(table):
    normalized = []
    for row in table:
        new_row = []
        for cell in row:
            cell = (cell or "").strip()
            if re.search(r'[a-zA-Z]', cell):
                new_row.append(cell)
            else:
                new_row.append(parse_number(cell))
        normalized.append(new_row)
    return normalized

def merge_multiline_rows(table):
    merged = []
    buffer = None
    for row in table:
        is_main_row = (row[0] != "") or any(re.search(r'\d', str(cell)) for cell in row[2:])
        if is_main_row:
            if buffer:
                merged.append(buffer)
            buffer = row[:]
        else:
            if buffer:
                buffer[1] += " " + row[1].strip()
    if buffer:
        merged.append(buffer)
    return merged

def get_table_text_set(table):
    """Lấy tất cả text từ bảng để loại bỏ"""
    text_set = set()
    for row in table:
        for cell in row:
            cell_text = str(cell).strip()
            if cell_text and cell_text != "None":
                # Thêm cả cell nguyên gốc và các từ riêng lẻ
                text_set.add(cell_text)
                # Tách thành các từ để matching tốt hơn
                words = cell_text.split()
                for word in words:
                    if len(word) > 1:  # Chỉ thêm từ có độ dài > 1
                        text_set.add(word)
    return text_set

def remove_table_text_from_page(page_text, table_texts_set):
    """Loại bỏ text của bảng khỏi page text"""
    lines = page_text.split("\n")
    filtered_lines = []

    for line in lines:
        line = line.strip()
        if not line:
            continue

        # Kiểm tra xem line có chứa text từ bảng không
        is_table_line = False

        # Kiểm tra exact match hoặc chứa nhiều từ từ bảng
        words_in_line = line.split()
        table_word_count = sum(1 for word in words_in_line if word in table_texts_set)

        # Nếu > 50% từ trong line là từ bảng thì coi như line này thuộc bảng
        if len(words_in_line) > 0 and table_word_count / len(words_in_line) > 0.5:
            is_table_line = True

        # Hoặc nếu line match exact với text trong bảng
        if line in table_texts_set:
            is_table_line = True

        if not is_table_line:
            filtered_lines.append(line)

    return "\n".join(filtered_lines)

output = {"pages": []}

with pdfplumber.open(pdf_path) as pdf:
    for page_index, page in enumerate(pdf.pages):
        page_data = {"page_number": page_index + 1, "text": "", "tables": []}

        # Extract text ban đầu
        text = page.extract_text() or ""

        # Extract tables
        tables = page.extract_tables({
            "vertical_strategy": "lines",
            "horizontal_strategy": "lines",
            "intersection_tolerance": 5,
        })

        if not tables:
            tables = page.extract_tables({
                "vertical_strategy": "text",
                "horizontal_strategy": "text",
                "snap_tolerance": 6,
                "join_tolerance": 6
            })

        # Process tables và thu thập text từ bảng
        all_table_texts = set()

        for t in tables:
            t = clean_table(t)
            t = remove_newlines_in_cells(t)

            # Thu thập text TRƯỚC KHI normalize số
            table_texts = get_table_text_set(t)
            all_table_texts.update(table_texts)

            # Sau đó mới xử lý số
            t = merge_multiline_rows(t)
            t = normalize_table_numbers(t)
            t = [row for row in t if any(str(cell).strip() for cell in row)]
            page_data["tables"].append(t)

        # Loại bỏ text của bảng khỏi page text
        if all_table_texts:
            text = remove_table_text_from_page(text, all_table_texts)

        page_data["text"] = "\n".join([line.strip() for line in text.split("\n") if line.strip()])
        output["pages"].append(page_data)

for page in output["pages"]:
    print(f"\nPAGE {page['page_number']}")

    print("\n- TEXT")
    print(page["text"])

    for _, table in enumerate(page["tables"]):
        print(f"\n- TABLE")
        for row in table:
            print("; ".join(str(cell) if str(cell).strip() != "" else "--" for cell in row))


PAGE 1

- TEXT
HÓA ĐƠN GIÁ TRỊ GIA TĂNG Ký hiệu(Serial):1C25MSN
(VAT INVOICE) Số(No.): 00000061
Ngày(Date)18tháng(month)10năm(year)2025
Mã CQT(Code):M1-25-MNZLR-00000001968
CÔNG TY TRÁCH NHIỆM HỮU HẠN CÔNG NGHỆ NƯỚNG YAKI
Mã số thuế(Tax code):0312533500
Địa chỉ(Address):375B Tân Sơn Nhì, Phường Phú Thọ Hòa, TP Hồ Chí Minh
Điện thoại(Tel):02838162188
Email:nhahangyaki@gmail.com
Số tài khoản(Bank account):
Họ và tên người mua hàng(Buyer):
Tên đơn vị(Company's name):CÔNG TY CỔ PHẦN GIẢI PHÁP MINH TÂM
MST/CCCD chủ hộ(Tax code/Household Head's ID):0317493763
Địa chỉ(Address):Số 20, đường số 22, khu dân cư Him Lam, ấp 4, Xã Bình Hưng, TP Hồ Chí Minh, Việt Nam
Điện thoại(Tel):
Hình thức thanh toán(Payment method):TM/CK
1/2

- TABLE
STT (No); Tên hàng hóa, dịch vụ (Name of goods and services); Đơn vị tính (Unit); Số lượng (Quantity); --; Đơn giá (Unit price); Thành tiền (Amount); --; Thuế suất GTGT (VAT rate); --; Tiền thuế GTGT (VAT Amount)
1; 7 Up; Lon; 2.0; --; 20000; 40000; --; 8%; --; 32